# Opening Lead

This is a tutorial for how to use the opening lead engine.

In [1]:
import os
import sys
sys.path.append('../../src')
os.environ['BEN_HOME'] = "../.."

from nn.models import Models
from bots import BotLead
from sample import Sample
import conf
import numpy as np

np.set_printoptions(precision=2, suppress=True, linewidth=200)
np.random.seed(42)


Instructions for updating:
non-resource variables are not supported in the long term


In [2]:
models = Models.from_conf(conf.load('../Conf/UCBC2024.conf'),'..')   # loading neural networks
sampler = Sample.from_conf(conf.load('../Conf/UCBC2024.conf'), False)  # Load sampling strategies

INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-bidding-3961000
INFO:tensorflow:Restoring parameters from ..\Models/NS1EW99-binfo-1887000
INFO:tensorflow:Restoring parameters from ..\Models/lead_suit-154000
INFO:tensorflow:Restoring parameters from ..\Models/lead_nt-59000
INFO:tensorflow:Restoring parameters from ..\Models/single_dummy-1562000
INFO:tensorflow:Restoring parameters from ..\Models/lefty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/dummy-920000
INFO:tensorflow:Restoring parameters from ..\Models/righty-1000000
INFO:tensorflow:Restoring parameters from ..\Models/decl-1000000


In [3]:
# both vulnerable. you are sitting North as dealer and you hold
hand = '5.A97643.Q82.QJ5'

# the auction goes:
auction = ["PAD_START", "PAD_START", "PAD_START", "1C", "PASS", "2S", "PASS", "3S", "PASS", "4D",  "PASS", "4S", "PASS", "4N", "PASS", "5D", "PASS", "6S", "PASS", "PASS", "PASS"]

# what to lead?

lead_bot = BotLead([True, True], hand, models, sampler, False)
lead = lead_bot.find_opening_lead(auction)

In [4]:
lead.card

HA

seems like the engine chose to lead the ace of diamonds

the other options it considered were: a small spade and a small club

In [5]:
lead.to_dict()['candidates']

[{'card': 'HA',
  'insta_score': 0.303,
  'expected_tricks_sd': 10.15,
  'p_make_contract': 0.1},
 {'card': 'CQ',
  'insta_score': 0.473,
  'expected_tricks_sd': 10.02,
  'p_make_contract': 0.07},
 {'card': 'Dx',
  'insta_score': 0.163,
  'expected_tricks_sd': 9.74,
  'p_make_contract': 0.04}]

in the above output:
- `insta_score` reflects the preference of the neural network
- `expected_tricks` how many tricks declarer is expected to take on this lead
- `p_make_contract` is the probability of the contract making on this lead

the probability of making and the expected tricks are computed on samples which are consistent with the auction. the samples are estimated single dummy using a neural network (more on this in another tutorial). we could also solve the samples double dummy, but that would be quite a bit slower.

In [6]:
# each row is one sample board
# the hands are in the order: LHO, Partner, RHO. Your cards are not shown as they are fixed/

lead.samples

['T9xx.KT8.x.AKT8x Jx.Qx.JT9xx.9xxx AKQ8xx.Jx.AKxx.x 0.28576',
 'A8xx.x.K9x.KT98x T9x.T8x.Txx.Axxx KQJxx.KQJ.AJxx.x 0.27957',
 '98x.x.Kxx.AKTxxx Jx.JT8x.JT9xxx.x AKQTxxx.KQ.A.98x 0.25140',
 'Txxx..Kxx.AKxxxx 98.T8xx.ATxx.98x AKQJxx.KQJ.J9x.T 0.24459',
 'Txx.x.K9x.AKT9xx Jxxx.JT8.JTxx.xx AKQ98.KQx.Axx.8x 0.21728',
 'JT9..KJxx.AK8xxx 8xx.J8xx.Txxx.xx AKQxxx.KQT.A9.T9 0.21333',
 'Txxx.xx.Kx.AKT8x Jxx.T8.JTxxx.9xx AKQ98.KQJ.A9x.xx 0.20875',
 'T9x.K8xx.x.AKT9x Jxx.JT.JT9xx.8xx AKQ8xx.Q.AKxx.xx 0.20628',
 'Txx.Kxx.9.AK9xxx 98x.QT8.KTxxx.Tx AKQJxx.J.AJxx.8x 0.20197',
 'JT8.QJT8.x.AKT9x xx.xx.AT9xxx.8xx AKQ9xxx.K.KJx.xx 0.19949']